# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [35]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

In [36]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

# Leemos instancias
df_optimo = pd.read_csv("..\\instances\\optimo\\indice.csv");
df_random_denso_denso_25 = pd.read_csv("..\\instances\\random-denso-denso-25\\indice.csv");
df_random_denso_denso_75 = pd.read_csv("..\\instances\\random-denso-denso-75\\indice.csv");
df_random_denso_ralo_25 = pd.read_csv("..\\instances\\random-denso-ralo-25\\indice.csv");
df_random_denso_ralo_75 = pd.read_csv("..\\instances\\random-denso-ralo-75\\indice.csv");
df_random_ralo_denso_25 = pd.read_csv("..\\instances\\random-ralo-denso-25\\indice.csv");
df_random_ralo_denso_75 = pd.read_csv("..\\instances\\random-ralo-denso-75\\indice.csv");
df_random_ralo_ralo_25 = pd.read_csv("..\\instances\\random-ralo-ralo-25\\indice.csv");
df_random_ralo_ralo_75 = pd.read_csv("..\\instances\\random-ralo-ralo-75\\indice.csv");

df_datasets = [
    df_random_denso_denso_25,
    df_random_denso_denso_75,
    df_random_denso_ralo_25,
    df_random_denso_ralo_75,
    df_random_ralo_denso_25,
    df_random_ralo_denso_75,
    df_random_ralo_ralo_25,
    df_random_ralo_ralo_75
]


In [37]:
def correr_experimento(metodo, archivo_instancia, longitud_tabu, tipo_memoria, porcentaje_vecindad,iteraciones,goloso_inicial):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)

    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../../main", metodo, str(longitud_tabu), tipo_memoria, str(porcentaje_vecindad),str(iteraciones),str(goloso_inicial)], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)
    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()
    algoritmo = metodo
    
    # Verificar que el proceso no fallo.
    assert exit_code == 0, F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}."
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    outputs = process.stderr.readline().split(); 
    tiempo_de_ejecucion = float(outputs[0]);
    impacto = int(outputs[1]);
    cant_colores = int(outputs[2]);
    longitud_tabu = int(outputs[3]);
    tipo_memoria = outputs[4];
    porcentaje_vecindad = int(outputs[5]);
    tiempo_mejor_solucion = int(outputs[6]);
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();

    return tiempo_de_ejecucion, impacto, cant_colores, longitud_tabu, tipo_memoria, porcentaje_vecindad, tiempo_mejor_solucion;

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [38]:
experimentos = [];

# Experimento 1

In [39]:
for df_dataset in df_datasets:
    for n in range(df_dataset.shape[0]):
        for goloso in ["GS","GE","GB"]:
            fila = df_dataset.iloc[n];
            experimentos.append([fila["dataset"],
                                 fila["n"],
                                 fila["mG"],
                                 fila["mH"],
                                 "TS",
                                 fila["archivo"],
                                 10,
                                 "soluciones",
                                 10,
                                 100,
                                 goloso,
                                 fila["interseccion"]])

In [ ]:
columnas = [
    "dataset",
    "n",
    "mG",
    "mH",
    "metodo",
    "tiempo",
    "impacto",
    "cant_colores",
    "longitud_tabu",
    "tipo_memoria",
    "porcentaje_vecindad",
    "iteraciones",
    "goloso_inicial",
    "interseccion",
    "tiempo_mejor_solucion"
];

filas = [];
numero = 1
T = 1 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    impactos = []
    cant_colores = []
    longitud_tabu = 0
    tipo_memoria = 0
    longitud_tabu = 0
    tipo_memoria = "none"
    porcentaje_vecindad = 0
    iteraciones = 0
    goloso_inicial = "GE"
    interseccion = ""
    tiempo_mejor_solucion = 0
    if len(experimento) > 6:
        longitud_tabu = experimento[6]
        tipo_memoria = experimento[7]
        porcentaje_vecindad = experimento[8]
        iteraciones = experimento[9]
        goloso_inicial = experimento[10]
        interseccion = experimento[11]
        
    tiempo, impacto, colores, longitud_tabu, tipo_memoria, porcentaje_vecindad, tiempo_mejor_solucion = correr_experimento(experimento[4],
                                                                                                        experimento[5],
                                                                                                        longitud_tabu,
                                                                                                        tipo_memoria,
                                                                                                        porcentaje_vecindad,
                                                                                                        iteraciones,
                                                                                                        goloso_inicial)
        
    tiempos.append(tiempo);
    impactos.append(impacto);
    cant_colores.append(colores);

    tiempo_promedio = np.median(tiempos);
    impacto_promedio = np.median(impactos);
    cant_colores_promedio = np.median(cant_colores);
    
    filas.append([
        experimento[0],
        experimento[1],
        experimento[2],
        experimento[3],
        experimento[4],
        tiempo_promedio,
        impacto_promedio,
        cant_colores_promedio,
        longitud_tabu,
        tipo_memoria,
        porcentaje_vecindad,
        iteraciones,
        goloso_inicial,
        interseccion,
        tiempo_mejor_solucion]
    );


'Experimento: 81/720'

In [ ]:

from os import listdir
from os.path import isfile, join
    
path = "../../instancias"; 
onlyfiles = [join(path,f) for f in listdir(path) if isfile(join(path, f)) and f.find(".out") != -1]

for i in range(0, df_optimo.shape[0]):
    fila = df_optimo.iloc[i];
    file = onlyfiles[i]
    with open(file) as f:
        coloring = int(next(f)) # read first line
        colors = [int(x) for x in next(f).split()]
        uniqueColors = len(set(colors))
        filas.append([fila["dataset"], fila["n"], fila["mG"], fila["mH"], "OPTIMO", 0, coloring, uniqueColors, 0, "none", 0, "",""]);


In [ ]:
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("../results/resultado.csv", index=False, header=True);